<a href="https://colab.research.google.com/github/Ronu2022/SPARK/blob/main/Apache_Spark/Pyspark%20/UDEMY_COURSE/2.RDD_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar -xzf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark
import os, findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
findspark.init()

^C

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .config("spark.python.worker.faulthandler.enabled", "true") \
    .config("spark.sql.execution.pyspark.udf.faulthandler.enabled", "true") \
    .master("local[*]") \
    .appName("RDD FROM A DATAFRAME EXAMPLE") \
    .getOrCreate()


sc = spark.sparkContext

FileNotFoundError: [Errno 2] No such file or directory: '/content/spark-3.5.0-bin-hadoop3/./bin/spark-submit'

In [ ]:
# FILE PATHS:
file_path_orders = "/content/drive/MyDrive/orders"
file_path_order_items= "/content/drive/MyDrive/order_items"

In [ ]:
rdd_orders = sc.textFile(file_path_orders)
rdd_order_items = sc.textFile(file_path_order_items)

In [ ]:
# using groupByKey   MapValues find the activity grouped by userid
rdd_logs = sc.parallelize([
    ("U001", "login"),
    ("U002", "watch_video"),
    ("U001", "logout"),
    ("U003", "login"),
    ("U002", "attempt_quiz"),
    ("U001", "watch_video")
])


In [ ]:
rdd_logs_agg = rdd_logs.groupByKey().mapValues(list)
print(rdd_logs_agg.collect())

[('U002', ['watch_video', 'attempt_quiz']), ('U003', ['login']), ('U001', ['login', 'logout', 'watch_video'])]


## **3.4 countByKey:**

In [ ]:
# Couint how many times each key appears in the RDD.

rdd_trades = sc.parallelize([
    ("AAPL", 100),
    ("GOOG", 200),
    ("AAPL", 150),
    ("MSFT", 300),
    ("GOOG", 250),
    ("AAPL", 50)
])


rdd_stocks_count = rdd_trades.countByKey()

print(rdd_stocks_count)

defaultdict(<class 'int'>, {'AAPL': 3, 'GOOG': 2, 'MSFT': 1})


In [ ]:
dict(rdd_stocks_count)

{'AAPL': 3, 'GOOG': 2, 'MSFT': 1}

# **4.SORTING:**

---



## **4.1 sortByKey:**

In [ ]:
rdd_trades = sc.parallelize([
    ("GOOG", 200),
    ("AAPL", 100),
    ("MSFT", 300),
    ("AAPL", 150),
    ("GOOG", 250)
])

rdd_sorted = rdd_trades.sortByKey()
print(rdd_sorted.collect())


In [ ]:
rdd_sorted = rdd_trades.sortByKey(ascending = False)
print(rdd_sorted.collect())

# **5.SET:**

---

## **4.1 union:**
- You have two RDDs of stock trades from two different exchanges. Combine them into one.


In [ ]:
rdd_nyse = sc.parallelize([
    ("AAPL", 100),
    ("MSFT", 200)
])

rdd_nasdaq = sc.parallelize([
    ("GOOG", 300),
    ("AAPL", 150)
])


rdd_union = rdd_nyse.union(rdd_nasdaq)
print(rdd_union.collect())

## **4.2 intersection():**

- finds the common records
- there must be an exact match


In [ ]:
rdd_nyse = sc.parallelize([
    ("AAPL", 100),
    ("MSFT", 200)
])

rdd_nasdaq = sc.parallelize([
    ("GOOG", 300),
    ("AAPL", 150)
])

# observe here AAPL 100 and AAPL 150 aren't the same
# Exected o/p: No Match

rdd_intersected = rdd_nyse.intersection(rdd_nasdaq)
print(rdd_intersected.collect())

In [ ]:
# Intersecting only by Keys:

rdd_nyse_keys = rdd_nyse.map(lambda x: x[0])
rdd_nasdaq_keys = rdd_nasdaq.map(lambda x: x[0])
rdd_keys_intersect = rdd_nyse_keys.intersection(rdd_nasdaq_keys)
print(rdd_keys_intersect.collect())


## **4.3 distinct():**

In [ ]:
rdd_trades = sc.parallelize([
    ("AAPL", 100),
    ("GOOG", 200),
    ("AAPL", 100),
    ("MSFT", 300),
    ("GOOG", 200),
    ("AAPL", 150)
])

rdd_trades_distinct = rdd_trades.distinct()
rdd_trades.take(10)

## **4.4 substract():**

In [ ]:
# Remove Cancelled Trades from all trades:

rdd_all_trades = sc.parallelize([
    ("AAPL", 100),
    ("GOOG", 200),
    ("MSFT", 300),
    ("AAPL", 150)
])

rdd_cancelled_trades = sc.parallelize([
    ("AAPL", 100),
    ("MSFT", 300)
])


rdd_current_trades_active = rdd_all_trades.subtract(rdd_cancelled_trades)

rdd_current_trades_active.take(10)